# The Contracting Standard Library
There are unique variables and functions available for smart contracts at runtime. In the last notebook, we used the ORM variables that are provided via the `stdlib` to interact with the database. We will explore the other methods in this notebook and the concept of `environment` and `ctx`.

To see the basic standard library included at runtime, use `gather()` from the `env` module.

In [3]:
from contracting.stdlib import env

In [4]:
env.gather()

{'Variable': contracting.db.orm.Variable,
 'Hash': contracting.db.orm.Hash,
 'ForeignVariable': contracting.db.orm.ForeignVariable,
 'ForeignHash': contracting.db.orm.ForeignHash,
 '__Contract': contracting.db.contract.Contract,
 'sha3': <function contracting.stdlib.bridge.hashing.sha3(hex_str:str)>,
 'sha256': <function contracting.stdlib.bridge.hashing.sha256(hex_str:str)>,
 'datetime': contracting.stdlib.bridge.time.Datetime}

All of these functions are available at runtime. You can extend the standard library by updating the `env` dictionary. If you want to pass something more dynamic through, like the time that a transaction was submitted to the network, you would pass in an environment object.

In [5]:
def stdlib_environment_examples():
    
    @export
    def sha3_data(s):
        return sha3(s)
    
    @export
    def return_env_variable():
        return this_will_be_defined_later

First, let's try to access the `sha3` function exposed in the `stdlib`.

In [6]:
from contracting.client import ContractingClient
client = ContractingClient(signer='stu')
client.submit(stdlib_environment_examples)
contract = client.get_contract('stdlib_environment_examples')

In [12]:
contract.sha3_data(s='00ff00ff00ff')

'a0d5f1e1000980a0ae98cffb12072a41328bbdfebf3f6012aa021b428daea5b7'

Now, if we try to call `return_env_variable()`, we will get an error. This is because the variable is not included in the `stdlib` nor has been defined elsewhere in the contract.

In [13]:
try:
    contract.return_env_variable()
except Exception as e:
    print(e)

name 'this_will_be_defined_later' is not defined


But if we pass it in the environment, it becomes accessible. This function is used by our blockchain to pass contextual information such as block height, block hash, transaction time, etc.

In [14]:
environment = {'this_will_be_defined_later': 42}
contract.return_env_variable(environment=environment)

42

## Runtime Context `ctx`
At runtime, there is a constant defined called `ctx`. `ctx` contains three fields:

* `ctx.signer`

    This is the signer of the initial transaction. This variable never changes. It should not be used for access control.

---

* `ctx.caller`
    
    This is the direct caller of the function. As explained in the next notebook, smart contracts can import functions from other smart contracts. If you submit a transaction to a smart contract which calls upon another smart contract, the call stack is modified. The `caller` becomes the calling smart contract on a function. This should be used for access control.
    
---    
    
* `ctx.this`

    This is the name of the smart contract. This is how you will reference a smart contract for ascribing ownership to them.

In [15]:
def call_me():
    @export
    def caller():
        return ctx.caller
    
    @export
    def this():
        return ctx.this

def ctx_example():
    import call_me
    
    @export
    def ctx_now():
        return ctx.signer, ctx.caller, ctx.this
    
    @export
    def ctx_after_call():
        c = call_me.caller()
        t = call_me.this()
        
        return ctx.signer, c, t

In [16]:
client.submit(call_me)
client.submit(ctx_example)

In [17]:
ctx_contract = client.get_contract('ctx_example')

In [18]:
ctx_contract.ctx_now()

('stu', 'stu', 'ctx_example')

In [19]:
ctx_contract.ctx_after_call()

('stu', 'ctx_example', 'call_me')

Notice above how `ctx_after_call` returns different information. This is because `ctx` is modified after each function call. Because `ctx_example` called `call_me`, the `ctx.caller` returned was `ctx_example`. If we call that function directly, we will get `stu` back.

In [20]:
call_me_contract = client.get_contract('call_me')

In [21]:
call_me_contract.caller()

'stu'

## Why does `ctx` being dynamic matter?
Having `ctx` lets you create smart contracts that act as operators for users and other smart contracts. Because they are given their own identity, they are essentially the signers of their own function calls. This allows you to give them their own accounts, balances, ownerships, etc. to create structures that behave complexly and securely.

Assume you have a bank smart contract. You want to keep everyone's balance inside of the main bank vault, but keep sub-accounts in their name. You don't want the bank to be able to spend someone else's money on their behalf, so you would check the `ctx.caller` to make sure it is the user, and not the bank itself.

Let's make a token contract to demonstrate this idea.

In [24]:
def token():
    balances = Hash()
    token_name = 'Stubucks'
    token_symbol = 'SBX'
    
    @construct
    def seed():
        # Whoever creates this smart contract is minted 1,000,000 tokens
        balances[ctx.caller] = 1000000
        
    @export
    def transfer(amount, to):
        assert balances[ctx.caller] >= amount, "You don't have enough to spend!"
        
        balances[ctx.caller] -= amount
        balances[to] += amount
        
    @export
    def allow(amount, spender):
        # We will explain multihashes in a bit!
        balances[ctx.caller, spender] = amount
        
    @export
    def spend_on_behalf(amount, owner, to):
        # We make sure we have a subaccount balance available
        assert balances[owner, ctx.caller] >= amount, "You can't spend that!"
        
        balances[owner, ctx.caller] -= amount
        
        balances[owner] -= amount
        balances[to] += amount
        
def bank():
    import token
    
    balances = Hash()
    
    @export
    def deposit(amount):
        token.spend_on_behalf(amount=amount, owner=ctx.caller, to=ctx.this)
        
        balances[ctx.caller] += amount
        
    @export
    def withdraw(amount):
        assert balances[ctx.caller] >= amount, "You don't have enough in your account!"
        
        balances[ctx.caller] -= amount
        
        token.transfer(amount=amount, to=ctx.spender)